In [9]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

*  Скачайте и поместите в папку ./Recommendation_Service следующие файлы:

[tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) — общая информация о фильмах

[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9) — информация о касте фильмов

In [10]:
df_movies = pd.read_csv('../../tmdb_5000_movies.csv')
df_credits = pd.read_csv('../../tmdb_5000_credits.csv')
df_credits.drop('title', axis= 1, inplace= True)
df_movies_1 = pd.merge(df_movies, df_credits, left_on= 'id', right_on= 'movie_id')
df_movies_1 = df_movies_1[df_movies.status == 'Released'].copy()
df_movies_1.dropna(subset= ['overview', 'genres', 'keywords'], inplace= True)
df_movies_1[['overview', 'genres', 'keywords']].fillna(' ', inplace= True)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_movies_1['overview'] = df_movies_1['overview'].fillna('')
df_movies_2 = df_movies_1['overview'] + ' ' + df_movies_1['keywords']

vector = TfidfVectorizer(stop_words= 'english', max_features= 10000)

matrix = vector.fit_transform(df_movies_2)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(matrix)

df_distance = pd.DataFrame(cosine_sim, index= df_movies_1['id'], columns= df_movies_1['id'])

In [13]:
folder_name = 'assets'

current_directory = os.getcwd()

parent_directory = os.path.dirname(current_directory)
new_folder_path = os.path.join(parent_directory, folder_name)

os.makedirs(new_folder_path, exist_ok=True)


In [14]:
df_distance.to_csv('../assets/distance.csv')
df_movies_1.to_csv('../assets/movies.csv')